In [96]:
# import modules
import tweepy
from tweepy.streaming import Stream
import pandas as pd
import numpy as np
import re
import os

# env module
import dotenv
dotenv_file = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_file, override=True)

# # TWITTER AUTHENTICATER # #
class TwitterAuthenticator():
    '''
    Class for keys authentication
    '''

    def __init__(self) -> None:
        pass

    def authenticate(self):
        # set online authentication
        auth = tweepy.OAuth1UserHandler(
            os.getenv('consumer_key'), 
            os.getenv('consumer_secret'), 
            os.getenv('access_token'), 
            os.getenv('access_token_secret'))

        # calling the API
        api = tweepy.API(auth)

        return api


# # TWEETS RAW DATA # #
class TweeterMining:
    '''
    Processing tweets searched by term
    '''

    df = pd.DataFrame()

    def __init__(self, search_term = None, tweets_amount=5):
        self.search_term = search_term
        self.tweets_amount = tweets_amount
        self.twitter_authenticator = TwitterAuthenticator()


    def get_tweets(self):
        '''
        gets informatio from the tweets, txt, ids, likes, and time.
        '''
        # use lowercases for search term
        search_term = self.search_term
        tweet_amount = self.tweets_amount
        api = self.twitter_authenticator.authenticate()
        tweets_raw = []
        ids = []
        likes = []
        time = []


        # return most recent search word
        # search by topic
        data = tweepy.Cursor(api.search_tweets,
                            q = search_term,
                            lang='en',
                            tweet_mode='extended',
                            ).items(tweet_amount)



        # getting data into a lists
        for tweet in data:
            #print(tweet.text)
            tweets_raw.append(tweet.full_text)
            ids.append(tweet.id)
            likes.append(tweet.favorite_count)
            time.append(tweet.created_at)

        df = pd.DataFrame({'tweets':tweets_raw, 'id':ids,'likes':likes,'time':time})

        # # removing retweets (all the line)
        df = df[~df.tweets.str.contains("RT")]

        self.df = df
        
        return df

    
    def cleaning(self): 
        '''
        removes extra information from the tweets
        '''  
        # cleaning tweets, removing mentioned users (@)
        self.df['tweets'] = self.df['tweets'].str.replace('htt.*', '', regex=True)
        self.df['tweets'] = self.df['tweets'].str.replace(r'@.\w+', '', regex=True)
        self.df['tweets'] = self.df['tweets'].str.replace(r'#.\w+', '', regex=True)
        self.df['tweets'] = self.df['tweets'].str.replace(r'\n', '', regex=True)
        self.df['tweets'] = self.df['tweets'].str.replace(u'[\U0001F600-\U0001F975].+', '', regex=True)

        # reset index
        self.df = self.df.reset_index(drop=True)

        return self.df








In [109]:
miningtweeter = TweeterMining("global warming ?", 20)

df = miningtweeter.get_tweets()

df = miningtweeter.cleaning()

# reset index
df = df.reset_index(drop=True)

df

,tweets,id,likes,time
0,54 years later Every landscape packed with buildings and y’all think global warming isn’t real?,1550872494789300225,0,2022-07-23 15:56:31+00:00
1,Global warming is fake news remember all the lies and predictions about it? Did they ever come true ??? 🐑 🐏 🐑,1550872182120517632,0,2022-07-23 15:55:16+00:00
2,Maybe consider global warming in Congress?,1550872147588829189,0,2022-07-23 15:55:08+00:00
3,One question to all you Climate Experts: Why did Global Warming switch to Climate Change? Feel free to elaborate.,1550871809846919168,1,2022-07-23 15:53:47+00:00
4,"Then the woman starts trying to give me a copy of their newspaper which has a headline that's just ""global warming"" so I ask her ""what, don't you believe it's happening?"" She does nonsense a few times and I insist she answers. Eventually ""Yes but because of all the 5g radiation""",1550871694075731971,0,2022-07-23 15:53:20+00:00
5,"First the Winter of Death now, the summer heatwave of global warming. What's next? The sky is falling? Hmm, better not make fun of that, it will probably be next",1550871117342146560,1,2022-07-23 15:51:02+00:00
6,"&lt; the same across every Earth?"" he wondered aloud. ""Does global warming on one Earth affect all Earths? Doesn't seem all that logical if it does.""",1550870939755491333,0,2022-07-23 15:50:20+00:00
7,So when in history...before a forestry dept. Did the native Americans suffer wild fires? Keep drinking the GOP coolaid. Global warming is real in the 80's and early 90's it never got this hot. Come back to this tweet when most of the us coasts are under water.,1550870789246926849,0,2022-07-23 15:49:44+00:00
8,"Truth. People fail to realize that climate is in constant state of change. Global warming is natural. Have we accelerated it? Yes. But, we will not stop it. Another ice age will inevitably happen. More government regulation to line their pockets will do absolutely nothing.",1550870737149378561,0,2022-07-23 15:49:32+00:00
9,91 lower Michigan yesterday Hot and humid today.Wonder how hot it would be if their was ACTUALLY GLOBAL WARMING?Republicans response for global warming “well we did have a ice age and it’s been warming up ever since”Ice age we didn’t have cars and factories,1550870706925232129,0,2022-07-23 15:49:24+00:00


In [122]:
test = []

for i in range(df.shape[0]):    
    test.append(i) 

df['test'] = test
df

,tweets,id,likes,time,test
0,54 years later Every landscape packed with buildings and y’all think global warming isn’t real?,1550872494789300225,0,2022-07-23 15:56:31+00:00,0
1,Global warming is fake news remember all the lies and predictions about it? Did they ever come true ??? 🐑 🐏 🐑,1550872182120517632,0,2022-07-23 15:55:16+00:00,1
2,Maybe consider global warming in Congress?,1550872147588829189,0,2022-07-23 15:55:08+00:00,2
3,One question to all you Climate Experts: Why did Global Warming switch to Climate Change? Feel free to elaborate.,1550871809846919168,1,2022-07-23 15:53:47+00:00,3
4,"Then the woman starts trying to give me a copy of their newspaper which has a headline that's just ""global warming"" so I ask her ""what, don't you believe it's happening?"" She does nonsense a few times and I insist she answers. Eventually ""Yes but because of all the 5g radiation""",1550871694075731971,0,2022-07-23 15:53:20+00:00,4
5,"First the Winter of Death now, the summer heatwave of global warming. What's next? The sky is falling? Hmm, better not make fun of that, it will probably be next",1550871117342146560,1,2022-07-23 15:51:02+00:00,5
6,"&lt; the same across every Earth?"" he wondered aloud. ""Does global warming on one Earth affect all Earths? Doesn't seem all that logical if it does.""",1550870939755491333,0,2022-07-23 15:50:20+00:00,6
7,So when in history...before a forestry dept. Did the native Americans suffer wild fires? Keep drinking the GOP coolaid. Global warming is real in the 80's and early 90's it never got this hot. Come back to this tweet when most of the us coasts are under water.,1550870789246926849,0,2022-07-23 15:49:44+00:00,7
8,"Truth. People fail to realize that climate is in constant state of change. Global warming is natural. Have we accelerated it? Yes. But, we will not stop it. Another ice age will inevitably happen. More government regulation to line their pockets will do absolutely nothing.",1550870737149378561,0,2022-07-23 15:49:32+00:00,8
9,91 lower Michigan yesterday Hot and humid today.Wonder how hot it would be if their was ACTUALLY GLOBAL WARMING?Republicans response for global warming “well we did have a ice age and it’s been warming up ever since”Ice age we didn’t have cars and factories,1550870706925232129,0,2022-07-23 15:49:24+00:00,9


In [114]:

def OpenAI(dataframe):

    replay = []
    client = tweepy.Client(
        os.getenv('bearer_token'), 
        os.getenv('consumer_key'), 
        os.getenv('consumer_secret'), 
        os.getenv('access_token'), 
        os.getenv('access_token_secret'))

    for i in range(df.shape[0]):
        temp = client.create_tweet(in_reply_to_tweet_id=dataframe.loc[i][1],text= dataframe.loc[i][4])
        replay.append(temp)

    return replay

In [115]:
replay = OpenAI(df)

In [117]:
print(replay)

None


In [120]:
client = tweepy.Client(
    os.getenv('bearer_token'), 
    os.getenv('consumer_key'), 
    os.getenv('consumer_secret'), 
    os.getenv('access_token'), 
    os.getenv('access_token_secret'))

client.create_tweet(text= df.loc[12][0])


Forbidden: 403 Forbidden
You are not allowed to create a Tweet with duplicate content.

Forbidden: 403 Forbidden
You are not allowed to create a Tweet with duplicate content.